In [17]:
from bs4 import BeautifulSoup
import pathlib, os
import networkx as nx
from pyvis.network import Network

# Netzwerkvisualisierung der in den Briefen erwähnten Personen

In [18]:
NOTEBOOK_PATH = pathlib.Path().resolve()
DATA_DIRECTORY = NOTEBOOK_PATH / "data" / "annotated"

Für das Netzwerk und Operationen im Netzwerk wird die Python-Bibliothek [networkx](https://networkx.org/) verwendet. Dazu wird zunächst ein Graph-Objekt erstellt, das anschließend die Knoten und Kanten vorhält. 

In [19]:
G = nx.Graph()

Knoten und Kanten kann man auf alle möglichen Arten bilden: Im folgenden Beispiel steht jedes ref-Attribut für einen Knoten, die Beschriftung bilden die verschiedenen Schreibweisen, die sich in TEI zwischen den Tags befinden. Eine Kante wird zwischen einer Personenreferenz und der nachfolgenden und vorhergehenden Referenz gebildet; es wird also davon ausgegagen, dass Personen in einem Brief etwas mit den vorher und nachher genannten Person zu tun haben. 

* Was wären andere Möglichkeiten Knoten zu bilden?
* Was wäre eine andere Variante, um Kanten zu bilden?

In [20]:
for FILE_NAME in os.listdir(DATA_DIRECTORY):
    if not FILE_NAME.endswith(".xml"):
        continue

    with open(DATA_DIRECTORY / FILE_NAME, "r", encoding="utf-8") as f:
        soup = BeautifulSoup(f, "xml")

    pers_names = soup.find_all("persName")
    previous_ref = None

    for pers in pers_names:
        ref = pers.get("ref")
        name = pers.get_text(strip=True)

        if not ref:
            continue  

        if not G.has_node(ref):
            G.add_node(ref, label=name, title=name, count=1, names=set())
        else:
            G.nodes[ref]["count"] += 1
            G.nodes[ref]["title"] += f", {name}"

        G.nodes[ref]["names"].add(name)

        if previous_ref and previous_ref != ref:
            G.add_edge(previous_ref, ref)

        previous_ref = ref

Für die Darstellung des Netzwerks in einer Visualisierung werden die Knoten von Bing, Labowsky und Klibansky rot einefärbt, um sie schneller identifizieren zu können. Außerdem wir die Größe der Knoten anhand der Häufigkeit der Nennungen (insgesamt skaliert).

* Wie könnte man die Knoten noch einfärben, um sich leichter in der Visualisierung orientieren zu können? 
* Welche Möglichkeit gibt es noch, um die Knotengröße festzulegen?

In [21]:
highlight_refs = {"gnd-1029912939", "gnd-118777378", "gnd-116183853"}  
for node_id in G.nodes:
    freq = G.nodes[node_id]["count"]
    G.nodes[node_id]["size"] = 10 + freq * 2  

    if node_id in highlight_refs:
        G.nodes[node_id]["color"] = "#FF0000"
    else:
        G.nodes[node_id]["color"] = "#97C2FC"

In [22]:
for node_id in G.nodes:
    names = G.nodes[node_id]["names"]
    label = ", ".join(sorted(names))
    G.nodes[node_id]["title"] = label
    G.nodes[node_id]["label"] = label
    del G.nodes[node_id]["names"]

Nachdem die Knoten vorverarbeitet wurden, erfolt hier ein Export in das `.gexf`-Format. Dieses Dateiformat lässt sich in [Gephi](https://gephi.org/) öffnen, einem WYSIWYG-Tool für die Analyse und Visualisierung von Netzwerken. 

In [23]:
nx.write_gexf(G, "brief-netzwerk.gexf")

Nachfolgend die eigentliche Visualisierung. Dazu wird die Bibliothek [pyvis](https://pyvis.readthedocs.io/en/latest/tutorial.html) verwendet und das Ergebnis in einer HTML-Datei gespeichert, die sich mit einem Browser öffnen lässt. 

In [24]:
net = Network(height="1000px", width="100%", notebook=True, bgcolor="#ffffff", font_color="black", cdn_resources='in_line')
net.from_nx(G)
net.force_atlas_2based()

net.show("brief-netzwerk.html")

brief-netzwerk.html
